In [1]:
import pandas as pd
import pickle
import utils

from_database = True

if from_database:
    # Load tables from database 
    dfs = utils.load_all_tables()
else:
    # Open the file in binary read mode and load the pickle data
    with open('preprocessed_data/dfs.pickle', 'rb') as f:
        dfs = pickle.load(f)


Found 46 tables: abuse_report_types, abuse_reports, ar_internal_metadata, articles, assertion_types, assertion_versions, assertions, assertions_genes, assertions_tags, assertions_technique_types, assertions_techniques, assessment_types, assessments, authors, banned_orcid_users, career_stages, claim_types, claims, comment_checks, comments, db_sets, ensembl_subdomains, expertise_levels, gene_names, gene_set_items, gene_sets, genes, journals, orcid_users, organisms, reason_types, reasons, rel_types, rels, schema_migrations, sessions, shares, statuses, tags, technique_types, techniques, tool_types, tools, users, workspace_orcid_users, workspaces
Loading abuse_report_types (4 rows)
Loading abuse_reports (0 rows)
Loading ar_internal_metadata (1 rows)
Loading articles (400 rows)
Loading assertion_types (10 rows)
Loading assertion_versions (19668 rows)
Loading assertions (13941 rows)
Loading assertions_genes (2053 rows)
Loading assertions_tags (403 rows)
Loading assertions_technique_types (0 r

In [2]:
if from_database:
    # Save tables to pickle
    with open('preprocessed_data/dfs.pickle', 'wb') as f:
        pickle.dump(dfs, f)

    # load it back with:
    with open('preprocessed_data/dfs.pickle', 'rb') as f:
        dfs = pickle.load(f)


 ## 2. Create dataframe claims with all the referenced table from the dataframes

In [3]:
import pickle
import pandas as pd
def clean_df(df):
    columns_to_remove = ['user_id', 'orcid_user_id', 
                    'created_at', 'updated_at', 'assertion_updated_at', 
                    'workspace_id', 'user_id', 'doi', 'organism_id', # 'pmid', 
                    'all_tags_json', 'obsolete', 'ext', 'badge_classes','pluralize_title',
                    'can_attach_file', 'refresh_side_panel', 'icon_classes', 'btn_classes']
    patterns_to_remove = ['validated', 'filename', 'obsolete_article']
    for col in columns_to_remove:
        if col in df.columns:
            df.drop(col, axis=1, inplace=True)
    for pattern in patterns_to_remove:
        cols = [c for c in df.columns if pattern in c]
        df.drop(cols, axis=1, inplace=True)
    return df



# preprocess: my unit here is the claims that are in assertion
# check wich columns have _id
claims = dfs["assertions"]
print(len(claims))
claims = claims[claims["obsolete"] == False]
print(len(claims))
claims = clean_df(claims)

id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)
# merge the article columsn
articles = clean_df(dfs["articles"])
articles = articles.rename(columns={"id": "article_id"})
# We keep article ID for later
claims = claims.merge(articles, on="article_id", how="left", suffixes=('', '_article'))#.drop("article_id", axis=1)


id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)

journals = clean_df(dfs["journals"])
journals = journals.drop('tag', axis=1).rename(columns={"id": "journal_id", "name": "journal_name"})
claims = claims.merge(journals, on="journal_id", how="left", suffixes=('', '_journal')).drop("journal_id", axis=1)

# same for assertion_type
assertion_types = clean_df(dfs["assertion_types"])
assertion_types = assertion_types.rename(columns={"id": "assertion_type_id", "name": "assertion_type"})
claims = claims.merge(assertion_types, on="assertion_type_id", how="left", suffixes=('', '_assertion_type')).drop("assertion_type_id", axis=1)

# same for assessment_type_id
assessment_types = clean_df(dfs["assessment_types"])
assessment_types = assessment_types.rename(columns={"id": "assessment_type_id", "name": "assessment_type"})
claims = claims.merge(assessment_types, on="assessment_type_id", how="left", suffixes=('', '_assessment_type')).drop("assessment_type_id", axis=1)

id_cols = [col for col in claims.columns if "_id" in col]
print(id_cols)


13941
13299
['article_id', 'assertion_type_id', 'assessment_type_id']
['article_id', 'assertion_type_id', 'assessment_type_id', 'journal_id']
['article_id']


/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_44193/3080443160.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(col, axis=1, inplace=True)
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_44193/3080443160.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(col, axis=1, inplace=True)
/var/folders/pj/15hxgl0j1wg2w_t5k7bvnq5c0000gn/T/ipykernel_44193/3080443160.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vie

In [4]:
claims = claims.drop(['published_at', 'badge_tag_classes','description', 'additional_context', 'references_txt'], axis=1) # most not consistently used accross dataset
claims = claims.set_index('id')

In [5]:
string_columns = claims.select_dtypes(include='object').columns
print(string_columns)

for col in string_columns:
    claims[col] = claims[col].apply(lambda x: x.replace('&amp;', '&') if isinstance(x, str) else x)

claims.to_csv('preprocessed_data/claims.csv')



Index(['content', 'pmid', 'authors_txt', 'title', 'volume', 'issue',
       'abstract', 'key', 'affs_json', 'large_scale', 'journal_name',
       'assertion_type', 'label', 'assessment_type'],
      dtype='object')


In [6]:
def truncate_string(s, max_length=20):
    """Truncate string to max_length characters."""
    if isinstance(s, str) and len(s) > max_length:
        return s[:max_length] + '...'
    return s

string_columns = string_columns.drop(["assessment_type"])

df_truncated = claims.copy()

for col in string_columns:
    if col in df_truncated.columns:
        df_truncated[col] = df_truncated[col].apply(lambda x: truncate_string(x))

# Save truncated dataframe
df_truncated.to_csv('preprocessed_data/claims_truncated_for_llm.csv', index=False)
df_truncated



,content,article_id,rank,pmid,authors_txt,title,pmid_article,volume,issue,abstract,...,large_scale,nber_tables,nber_panels,journal_name,impact_factor,assertion_type,label,is_assessed,assessment_type,rank_assessment_type
id,,,,,,,,,,,,,,,,,,,,,
5838,"<p>Belozerov VE, Lin...",2049.0,1,None,Zhuang ZH;Zhou Y;Yu ...,Regulation of Drosop...,16014325.0,18,4,The p38 mitogen-acti...,...,False,0.0,14.0,Cellular signalling,4.8,reference,Reference,False,NaN,NaN
693,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,False,NaN,NaN,"Science (New York, N...",56.9,assessment,Assessment,False,Not assessed,13.0
695,,2517.0,1,None,Choe KM;Werner T;Stö...,Requirement for a pe...,11872802.0,296,5566,Components of microb...,...,False,NaN,NaN,"Science (New York, N...",56.9,assessment,Assessment,False,Not assessed,13.0
5840,Verified by many sub...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,False,2.0,9.0,Proceedings of the N...,11.1,assessment,Assessment,False,Verified,1.0
5839,Activation of a cell...,2204.0,1,None,Zettervall CJ;Anderl...,A directed screen fo...,15381778.0,101,39,An attack by a paras...,...,False,2.0,9.0,Proceedings of the N...,11.1,main_claim,Main claim,True,Verified,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14468,The DIF antiserum us...,2898.0,1,None,Williams MJ;Rodrigue...,The 18-wheeler mutat...,9321392.0,16,20,Mammals and insects ...,...,False,0.0,27.0,The EMBO journal,11.4,comment,Comment,False,Not assessed,13.0
11330,Although independent...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,False,1.0,27.0,Journal of cell scie...,4.0,assessment,Assessment,False,"Unchallenged, logically consistent",6.0
11329,Spatzle (#gene:FBgn0...,1577.0,1,None,Shia AK;Glittenberg ...,Toll-dependent antim...,19934223.0,122,Pt 24,"In Drosophila, the h...",...,False,1.0,27.0,Journal of cell scie...,4.0,main_claim,Main claim,True,"Unchallenged, logically consistent",6.0
